# Create Grappa dataset

## 1. Create top-files from pdb-files

In [ ]:
%%bash 

# make sure to have a running version of gromacs for this code block

top_dir="gmx_top"
pdb_dir="pdbs"

cwd=$(pwd)

cp -r $pdb_dir $top_dir
cp -r amber99sb-ildn.ff $top_dir 

pushd  $top_dir &>> /dev/null

for dir in *; do
    if [[ "$(basename "$dir")" == "amber99sb-ildn.ff" ]]; then
        continue  
    fi   

    pushd $dir &>> /dev/null
    ln -s ../amber99sb-ildn.ff amber99sb-ildn.ff
    printf "1\n1\n " | gmx pdb2gmx -f pep.pdb -o pep.gro -p pep.top -i pep.itp -n pep.ndx -ignh -nov &>> /dev/null
    gmx editconf -f pep.gro -o pep.gro -c -d 1.0 &>> /dev/null   # add simulation box
    popd &>> /dev/null
    # rm "$pdb"
done 

popd &>> /dev/null

### 1.1 Check whether GROMACS topology files were written to the target directory

In [ ]:
%%bash 

parent_dir="gmx_top"

# Initialize counters
count=0
total=-1 #substract force field dir

# Loop through subdirectories
for subdir in "$parent_dir"/*/; do
    total=$((total + 1))  # Increment total subdirectory counter

    # Check if both .pdb and .top files exist in the subdirectory
    if ls "$subdir"/*.pdb 1> /dev/null 2>&1 && ls "$subdir"/*.top 1> /dev/null 2>&1; then
        count=$((count + 1))
    fi
done

# Print the result
echo "Number of subdirectories in $parent_dir with both .pdb and .top files: $count"
echo "Total number of subdirectories: $total"

## 1.2 Create a subset of all pdbs/tops for easier handling

In [21]:
%%bash
# Define source and target directories
source_dir="gmx_top"
target_dir="tutorial-MD-input"

# Ensure the target directory exists
mkdir -p "$target_dir"

# Copy subdirectories starting with 'G'
for subdir in "$source_dir"/G*/; do
    if [ -d "$subdir" ]; then
        cp -r "$subdir" "$target_dir"
    fi
done

cp -r $source_dir/amber99sb-ildn.ff $target_dir


## 2. Sample States with MD

## 2.0 Set up a python environment with OpenMM and Grappa

In [ ]:
%%bash
# not tested, check https://github.com/graeter-group/grappa?tab=readme-ov-file#installation for the most recent installation instructions
conda create -n grappa python=3.10 -y
conda activate grappa

conda install -c conda-forge openmm # optional: cudatoolkit=<YOUR CUDA>
python -m openmm.testInstallation   # verify that OpenMM can run on your GPU platform

git clone https://github.com/hits-mbm-dev/grappa.git
cd grappa

pip install -r installation/cpu_requirements.txt
pip install -e .
python tests/test_installation.py   # verify that your Grappa installation is running

## 2.1 Sample States

In [ ]:
%%bash

# For this step, a python environment with OpenMM and Grappa is required
# Grappa imports may take a while

python ../generate_states.py tutorial-MD-input -n 10 -t 100 --t_max 500 --gmx_topology

## 3. Calculate QM single point energies and forces

## 3.0 Set up a python environment with Psi4 and ASE

In [ ]:
%%bash
# not tested, check out https://psicode.org/psi4manual/master/build_obtaining.html#faq-binarypackage and https://wiki.fysik.dtu.dk/ase/install.html

conda create -n p4env psi4 -c conda-forge/label/libint_dev -c conda-forge
conda activate p4env
psi4 --test

pip install --upgrade ase

## 3.1 Calculate single points

In [ ]:
%%bash

# For this step, a python environment with psi4 and ase is required
# These calculations should be run on a compute cluster with the appropriate settings

THIS_DIR=$(pwd)
DS="tutorial-MD-input"

MEM=32
CORES=6


python ../single_points.py "$THIS_DIR/$DS" -m $MEM -t $CORES -s &

## 4. Create a Grappa dataset

In [ ]:
# Here, a python environment with OpenMM, ASE and Grappa is required
# You can use the supplied QM data to continue without having to calculate it yourself

from pathlib import Path
from grappa.data.dataset_builder import DatasetBuilder

cwd = Path().cwd()

dataset_qm = cwd / "tutorial-QM-input"
dataset_top = cwd / "tutorial-MD-input"
db = DatasetBuilder.from_QM_dicts(dataset_qm,verbose=True)
db.add_nonbonded_from_gmx_top(dataset_top,add_pdb=True)
db.remove_bonded_parameters()
db.filter_bad_nonbonded()
db.write_to_dir(cwd / 'tutorial-dataset',overwrite=True,delete_dgl=True)

## 5. Inspect the finished dataset

In [ ]:
%%bash 
# This requires an environment with Grappa
grappa_inspect-dataset tutorial_dataset

You should see something like this:

```
Dataset: tutorial-dataset with 20 molecules and 998 conformations

QM Energy mean: -0.00, std:  5.96, max: 20.08, min: -17.58 [kcal/mol]
QM Gradient norm mean: 36.28, std: 23.23, max: 182.62 [kcal/mol/Å]
Target energy difference (QM - MM(nonbonded)) for reference_ff: mean -0.00, std:  6.73, max: 20.13, min -22.99 [kcal/mol]
Target gradient norm difference (QM - MM(nonbonded)) for reference_ff: mean 31.26, std:  4.14, max: 48.41, min 16.91 [kcal/mol/A]
Structures
xyz: 20	pdb: 20	

QM data
energy: 20	gradient: 20	

FF Parameters
bond_eq: 0	bond_k: 0	
angle_k: 0	angle_eq: 0	
proper_ks: 0	proper_phases: 0	
improper_ks: 0	improper_phases: 0	

FF Energy/Gradients
qm energy total: 20; gradient total: 20
reference_ff energy total: 20, nonbonded: 20; gradient total: 20, nonbonded: 20
```

## 6. Train on the finished dataset

In [ ]:
%%bash

# A grappa environment, ideally with gpu support, is required for training 

cp -r tutorial-dataset/ <grappa_src_dir>/data/datasets/
python experiments/train.py experiment=grappa-continued data=grappa-from-ckpt data.extra_datasets=[tutorial-dataset]

You should have a link to your wandb run in stdout and see the progress. After finishing, the accuracy on the test set should be roughly

```
[2024-12-16 10:53:25,961][root][INFO] - Test summary:
                 n_mols n_confs rmse_energies crmse_gradients
tutorial-dataset      2     100    1.77+-0.05      5.32+-0.02
```